In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from dateutil import tz
import json
import ccxt

In [2]:
def convert_tz(utc):
    from_zone = tz.tzutc()
    to_zone = tz.tzlocal()
    utc = utc.replace(tzinfo=from_zone).astimezone(to_zone)
    
    return utc

### Config 

In [3]:
with open('_keys/binance_keys.json') as keys_file:
    keys_dict = json.load(keys_file)
exchange = ccxt.binance({'apiKey': keys_dict['apiKey'],
                         'secret': keys_dict['secret'],
                         'enableRateLimit': True,
                         'options': {'newOrderRespType': 'FULL'}})

In [4]:
symbol = 'LUNA/USDT'
timeframe = '15m'
since = None
limit = 100

### Ticker

In [5]:
ticker = exchange.fetch_ticker(symbol)
latest_price = ticker['last']

In [6]:
latest_price

17.358

### OHLCV

In [7]:
ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)

In [8]:
df = pd.DataFrame(ohlcv)
df.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
df['Time'] = pd.to_datetime(df['Time'], unit = 'ms')
df['Time'] = df['Time'].apply(lambda x: convert_tz(x))

In [9]:
df

,Time,Open,High,Low,Close,Volume
0,2021-04-27 23:15:00+07:00,17.771,17.937,17.700,17.744,146241.937
1,2021-04-27 23:30:00+07:00,17.738,17.839,17.572,17.778,127317.221
2,2021-04-27 23:45:00+07:00,17.778,17.781,17.502,17.570,81678.206
3,2021-04-28 00:00:00+07:00,17.578,17.657,17.423,17.474,173577.111
4,2021-04-28 00:15:00+07:00,17.464,17.649,17.450,17.634,133399.785
...,...,...,...,...,...,...
95,2021-04-28 23:00:00+07:00,17.290,17.347,17.262,17.309,67920.358
96,2021-04-28 23:15:00+07:00,17.307,17.334,17.277,17.308,38770.432
97,2021-04-28 23:30:00+07:00,17.301,17.378,17.301,17.377,65912.654
98,2021-04-28 23:45:00+07:00,17.377,17.386,17.343,17.347,28901.161


### Order

In [10]:
open_orders = exchange.fetch_open_orders(symbol)

In [11]:
open_orders

[{'info': {'symbol': 'LUNAUSDT',
   'orderId': '151319979',
   'orderListId': '-1',
   'clientOrderId': 'web_a20c4727bf1c4d9d892fe83326533400',
   'price': '16.00000000',
   'origQty': '25.00000000',
   'executedQty': '0.00000000',
   'cummulativeQuoteQty': '0.00000000',
   'status': 'NEW',
   'timeInForce': 'GTC',
   'type': 'LIMIT',
   'side': 'BUY',
   'stopPrice': '0.00000000',
   'icebergQty': '0.00000000',
   'time': '1619543422140',
   'updateTime': '1619543422140',
   'isWorking': True,
   'origQuoteOrderQty': '0.00000000'},
  'id': '151319979',
  'clientOrderId': 'web_a20c4727bf1c4d9d892fe83326533400',
  'timestamp': 1619543422140,
  'datetime': '2021-04-27T17:10:22.140Z',
  'lastTradeTimestamp': None,
  'symbol': 'LUNA/USDT',
  'type': 'limit',
  'timeInForce': 'GTC',
  'postOnly': False,
  'side': 'buy',
  'price': 16.0,
  'stopPrice': 0.0,
  'amount': 25.0,
  'cost': 0.0,
  'average': None,
  'filled': 0.0,
  'remaining': 25.0,
  'status': 'open',
  'fee': None,
  'trades':

In [12]:
order_type = 'limit'
side = 'buy'
price = latest_price / 3
amount = 20 / price

order = exchange.create_order(symbol, order_type, side, amount, price)

In [13]:
order

{'info': {'symbol': 'LUNAUSDT',
  'orderId': '152574166',
  'orderListId': '-1',
  'clientOrderId': 'x-R4BD3S82292f369a9a6eb5e439cc60',
  'transactTime': '1619630067869',
  'price': '5.78600000',
  'origQty': '3.45600000',
  'executedQty': '0.00000000',
  'cummulativeQuoteQty': '0.00000000',
  'status': 'NEW',
  'timeInForce': 'GTC',
  'type': 'LIMIT',
  'side': 'BUY'},
 'id': '152574166',
 'clientOrderId': 'x-R4BD3S82292f369a9a6eb5e439cc60',
 'timestamp': 1619630067869,
 'datetime': '2021-04-28T17:14:27.869Z',
 'lastTradeTimestamp': None,
 'symbol': 'LUNA/USDT',
 'type': 'limit',
 'timeInForce': 'GTC',
 'postOnly': False,
 'side': 'buy',
 'price': 5.786,
 'stopPrice': None,
 'amount': 3.456,
 'cost': 0.0,
 'average': None,
 'filled': 0.0,
 'remaining': 3.456,
 'status': 'open',
 'fee': None,
 'trades': [],
 'fees': []}

In [14]:
open_orders = exchange.fetch_open_orders(symbol)

In [15]:
open_orders

[{'info': {'symbol': 'LUNAUSDT',
   'orderId': '151319979',
   'orderListId': '-1',
   'clientOrderId': 'web_a20c4727bf1c4d9d892fe83326533400',
   'price': '16.00000000',
   'origQty': '25.00000000',
   'executedQty': '0.00000000',
   'cummulativeQuoteQty': '0.00000000',
   'status': 'NEW',
   'timeInForce': 'GTC',
   'type': 'LIMIT',
   'side': 'BUY',
   'stopPrice': '0.00000000',
   'icebergQty': '0.00000000',
   'time': '1619543422140',
   'updateTime': '1619543422140',
   'isWorking': True,
   'origQuoteOrderQty': '0.00000000'},
  'id': '151319979',
  'clientOrderId': 'web_a20c4727bf1c4d9d892fe83326533400',
  'timestamp': 1619543422140,
  'datetime': '2021-04-27T17:10:22.140Z',
  'lastTradeTimestamp': None,
  'symbol': 'LUNA/USDT',
  'type': 'limit',
  'timeInForce': 'GTC',
  'postOnly': False,
  'side': 'buy',
  'price': 16.0,
  'stopPrice': 0.0,
  'amount': 25.0,
  'cost': 0.0,
  'average': None,
  'filled': 0.0,
  'remaining': 25.0,
  'status': 'open',
  'fee': None,
  'trades':

In [16]:
order = exchange.cancel_order(order['id'], symbol)

In [17]:
order

{'info': {'symbol': 'LUNAUSDT',
  'origClientOrderId': 'x-R4BD3S82292f369a9a6eb5e439cc60',
  'orderId': '152574166',
  'orderListId': '-1',
  'clientOrderId': 'rJuJcClY5gLk0A1AymOvzP',
  'price': '5.78600000',
  'origQty': '3.45600000',
  'executedQty': '0.00000000',
  'cummulativeQuoteQty': '0.00000000',
  'status': 'CANCELED',
  'timeInForce': 'GTC',
  'type': 'LIMIT',
  'side': 'BUY'},
 'id': '152574166',
 'clientOrderId': 'rJuJcClY5gLk0A1AymOvzP',
 'timestamp': None,
 'datetime': None,
 'lastTradeTimestamp': None,
 'symbol': 'LUNA/USDT',
 'type': 'limit',
 'timeInForce': 'GTC',
 'postOnly': False,
 'side': 'buy',
 'price': 5.786,
 'stopPrice': None,
 'amount': 3.456,
 'cost': 0.0,
 'average': None,
 'filled': 0.0,
 'remaining': 3.456,
 'status': 'canceled',
 'fee': None,
 'trades': [],
 'fees': []}

In [18]:
open_orders = exchange.fetch_open_orders(symbol)

In [19]:
open_orders

[{'info': {'symbol': 'LUNAUSDT',
   'orderId': '151319979',
   'orderListId': '-1',
   'clientOrderId': 'web_a20c4727bf1c4d9d892fe83326533400',
   'price': '16.00000000',
   'origQty': '25.00000000',
   'executedQty': '0.00000000',
   'cummulativeQuoteQty': '0.00000000',
   'status': 'NEW',
   'timeInForce': 'GTC',
   'type': 'LIMIT',
   'side': 'BUY',
   'stopPrice': '0.00000000',
   'icebergQty': '0.00000000',
   'time': '1619543422140',
   'updateTime': '1619543422140',
   'isWorking': True,
   'origQuoteOrderQty': '0.00000000'},
  'id': '151319979',
  'clientOrderId': 'web_a20c4727bf1c4d9d892fe83326533400',
  'timestamp': 1619543422140,
  'datetime': '2021-04-27T17:10:22.140Z',
  'lastTradeTimestamp': None,
  'symbol': 'LUNA/USDT',
  'type': 'limit',
  'timeInForce': 'GTC',
  'postOnly': False,
  'side': 'buy',
  'price': 16.0,
  'stopPrice': 0.0,
  'amount': 25.0,
  'cost': 0.0,
  'average': None,
  'filled': 0.0,
  'remaining': 25.0,
  'status': 'open',
  'fee': None,
  'trades':

In [20]:
closed_order = exchange.fetch_closed_orders(symbol)

In [21]:
closed_order

[{'info': {'symbol': 'LUNAUSDT',
   'orderId': '141612146',
   'orderListId': '-1',
   'clientOrderId': 'and_3b0684b6248f41349b5c24de5b06cbdf',
   'price': '13.63810000',
   'origQty': '49.16000000',
   'executedQty': '49.16000000',
   'cummulativeQuoteQty': '670.44899600',
   'status': 'FILLED',
   'timeInForce': 'GTC',
   'type': 'LIMIT',
   'side': 'BUY',
   'stopPrice': '0.00000000',
   'icebergQty': '0.00000000',
   'time': '1619069139146',
   'updateTime': '1619069195122',
   'isWorking': True,
   'origQuoteOrderQty': '0.00000000'},
  'id': '141612146',
  'clientOrderId': 'and_3b0684b6248f41349b5c24de5b06cbdf',
  'timestamp': 1619069139146,
  'datetime': '2021-04-22T05:25:39.146Z',
  'lastTradeTimestamp': None,
  'symbol': 'LUNA/USDT',
  'type': 'limit',
  'timeInForce': 'GTC',
  'postOnly': False,
  'side': 'buy',
  'price': 13.6381,
  'stopPrice': 0.0,
  'amount': 49.16,
  'cost': 670.448996,
  'average': 13.6381,
  'filled': 49.16,
  'remaining': 0.0,
  'status': 'closed',
  '

In [22]:
test_id = '151319398'
test_order = exchange.fetch_order(test_id, symbol)

In [23]:
test_order

{'info': {'symbol': 'LUNAUSDT',
  'orderId': '151319398',
  'orderListId': '-1',
  'clientOrderId': 'web_464dea3250fe4eeb9f92782d476957aa',
  'price': '17.00000000',
  'origQty': '25.00000000',
  'executedQty': '25.00000000',
  'cummulativeQuoteQty': '425.00000000',
  'status': 'FILLED',
  'timeInForce': 'GTC',
  'type': 'LIMIT',
  'side': 'BUY',
  'stopPrice': '0.00000000',
  'icebergQty': '0.00000000',
  'time': '1619543389590',
  'updateTime': '1619585307662',
  'isWorking': True,
  'origQuoteOrderQty': '0.00000000'},
 'id': '151319398',
 'clientOrderId': 'web_464dea3250fe4eeb9f92782d476957aa',
 'timestamp': 1619543389590,
 'datetime': '2021-04-27T17:09:49.590Z',
 'lastTradeTimestamp': None,
 'symbol': 'LUNA/USDT',
 'type': 'limit',
 'timeInForce': 'GTC',
 'postOnly': False,
 'side': 'buy',
 'price': 17.0,
 'stopPrice': 0.0,
 'amount': 25.0,
 'cost': 425.0,
 'average': 17.0,
 'filled': 25.0,
 'remaining': 0.0,
 'status': 'closed',
 'fee': None,
 'trades': [],
 'fees': []}

In [24]:
status = test_order['info']['status'] # use info status not global status

In [25]:
status

'FILLED'

### Balance

In [26]:
balance = exchange.fetch_balance()

In [27]:
pd.DataFrame(balance['info']['balances'])

,asset,free,locked
0,BTC,0.00000000,0.00000000
1,LTC,0.00000000,0.00000000
2,ETH,0.00000221,0.00000000
3,NEO,0.00000000,0.00000000
4,BNB,0.00000000,0.00000000
...,...,...,...
407,1INCHUP,0.00000000,0.00000000
408,1INCHDOWN,0.00000000,0.00000000
409,MIR,0.00000000,0.00000000
410,BAR,0.00000000,0.00000000


In [28]:
coin = symbol.split('/')[0]
balance[coin]

{'free': 24.97584, 'used': 0.0, 'total': 24.97584}